In [5]:
import numpy as np
import pandas as pd
import requests
import seaborn as sn
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import time

## Web Scraping

In [3]:
url = 'https://fortune.com/ranking/global500/'

In [5]:
requests.get(url)

<Response [200]>

In [7]:
request_header = {'content':'https://fortune.com/ranking/fortune500/'}

In [11]:
requests.get(url, headers = request_header)

<Response [200]>

In [13]:
page = requests.get(url, headers = request_header)
pagecontent = page.text
pagecontent

'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Fortune Global 500 – The largest companies in the world by revenue | Fortune</title><meta name="msapplication-TileColor" content="#ffc40d"/><meta name="theme-color" content="#FFFFFF"/><meta name="description" content="The corporations on our annual list posted aggregate revenues of $41 trillion for the second year in a row."/><meta name="keywords"/><meta property="og:url" content="https://fortune.com/ranking/global500/"/><meta property="og:type" content="article"/><meta property="og:title" content="Fortune Global 500"/><meta property="og:description" content="The corporations on our annual list posted aggregate revenues of $41 trillion for the second year in a row."/><meta property="og:site_name" content="Fortune"/><meta property="og:image" content="https://fortune.com/img-assets/wp-content/uploads/2023/08/Fortune-Global-500-logo-e1690922780643.jpg?resize=1200,600"/><

In [15]:
soup = BeautifulSoup(pagecontent)
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Fortune Global 500 – The largest companies in the world by revenue | Fortune</title><meta content="#ffc40d" name="msapplication-TileColor"/><meta content="#FFFFFF" name="theme-color"/><meta content="The corporations on our annual list posted aggregate revenues of $41 trillion for the second year in a row." name="description"/><meta name="keywords"/><meta content="https://fortune.com/ranking/global500/" property="og:url"/><meta content="article" property="og:type"/><meta content="Fortune Global 500" property="og:title"/><meta content="The corporations on our annual list posted aggregate revenues of $41 trillion for the second year in a row." property="og:description"/><meta content="Fortune" property="og:site_name"/><meta content="https://fortune.com/img-assets/wp-content/uploads/2023/08/Fortune-Global-500-logo-e1690922780643.jpg?resize=1200,600" property="og:image"/><

In [17]:
years = soup.find_all('span',attrs={'class':'item current placeholder'})
years

[<span class="item current placeholder">2024</span>]

In [19]:
year = soup.find('span', attrs={'class': 'item current placeholder'})
if year:
    print(year.text.strip())
else:
    print("Year element not found!")

2024


In [21]:
# Column Names
column_names = []
for company in soup.find_all('table',attrs={'role':'table'}):
    column_name = company.find('tr',attrs={'role':'row'})
    if column_name is None:
        column_names.append(np.NaN)

    else:
        column_names.append(column_name.text.strip())
print(column_names)

['RemoveRankRemoveNameRemoveRevenues ($M)RemoveRevenue Percent ChangeRemoveProfits ($M)RemoveProfits Percent ChangeRemoveAssets ($M)RemoveEmployeesRemoveChange in RankRemoveYears on Global 500 List']


In [23]:
# List of Column Names
column_names = []
for company in soup.find_all('table',attrs={'role':'table'}):
    column_name = company.find('tr',attrs={'role':'row'})
    if column_name:
        titles = [row.text.strip() for row in column_name]
        column_names.append(titles)
    else:
        # Append a row of NaNs if no data is found
        column_names.append([np.NaN] * len(column_name))

print(column_names)

[['RemoveRank', 'RemoveName', 'RemoveRevenues ($M)', 'RemoveRevenue Percent Change', 'RemoveProfits ($M)', 'RemoveProfits Percent Change', 'RemoveAssets ($M)', 'RemoveEmployees', 'RemoveChange in Rank', 'RemoveYears on Global 500 List']]


In [25]:
headers = ['RemoveRank', 'RemoveName', 'RemoveRevenues ($M)', 'RemoveRevenue Percent Change', 'RemoveProfits ($M)', 'RemoveProfits Percent Change', 
           'RemoveAssets ($M)', 'RemoveEmployees', 'RemoveChange in Rank', 'RemoveYears on Global 500 List']

# Cleaned headers (removing 'Remove')
cleaned_headers = [header.replace('Remove', '').strip() for header in headers]
cleaned_headers

['Rank',
 'Name',
 'Revenues ($M)',
 'Revenue Percent Change',
 'Profits ($M)',
 'Profits Percent Change',
 'Assets ($M)',
 'Employees',
 'Change in Rank',
 'Years on Global 500 List']

In [27]:
# 1-500 companies and their details in the form of text

for company in soup.find_all('table',attrs={'role':'table'}):
    details = company.find('tbody',attrs={'role':'rowgroup'})
    if details is None:
        print(np.NaN)
        print()
    else:
        print(details.text)

1Walmart$648,1256%$15,51132.8%$252,3992,100,000-30View More details about Walmart2Amazon$574,78511.8%$30,425-$527,8541,525,000216View More details about Amazon3State Grid$545,947.53%$9,204.312.4%$781,126.21,361,423-24View More details about State Grid4Saudi Aramco$494,890.1-18%$120,699.3-24.1%$660,819.273,311-26View More details about Saudi Aramco5Sinopec Group$429,699.7-8.8%$9,393.4-2.7%$382,688513,434126View More details about Sinopec Group6China National Petroleum$421,713.6-12.7%$21,294.71%$630,5621,026,301-124View More details about China National Petroleum7Apple$383,285-2.8%$96,995-2.8%$352,583161,000122View More details about Apple8UnitedHealth Group$371,62214.6%$22,38111.2%$273,720440,000228View More details about UnitedHealth Group9Berkshire Hathaway$364,48220.7%$96,223-$1,069,978396,500528View More details about Berkshire Hathaway10CVS Health$357,77611%$8,344101.1%$249,728259,500129View More details about CVS Health11Volkswagen$348,408.118.6%$17,944.510.5%$663,063.8684,025430V

In [29]:
# Companies data separated by list for individual company

companies_data = []
for company1 in soup.find_all('tbody',attrs={'role':'rowgroup'}):
    for company2 in company1.find_all('tr',attrs={'role':'row'}):
        details1 = company2.find_all('td',attrs={'role':'cell','data-cy': 'list-cell'})

        if details1:
            row_data = [cell.text.strip() for cell in details1]
            print(row_data)

        else:
            listb1.append([np.NaN])
# print(companies_data)

['1', 'Walmart', '$648,125', '6%', '$15,511', '32.8%', '$252,399', '2,100,000', '-', '30']
['2', 'Amazon', '$574,785', '11.8%', '$30,425', '-', '$527,854', '1,525,000', '2', '16']
['3', 'State Grid', '$545,947.5', '3%', '$9,204.3', '12.4%', '$781,126.2', '1,361,423', '-', '24']
['4', 'Saudi Aramco', '$494,890.1', '-18%', '$120,699.3', '-24.1%', '$660,819.2', '73,311', '-2', '6']
['5', 'Sinopec Group', '$429,699.7', '-8.8%', '$9,393.4', '-2.7%', '$382,688', '513,434', '1', '26']
['6', 'China National Petroleum', '$421,713.6', '-12.7%', '$21,294.7', '1%', '$630,562', '1,026,301', '-1', '24']
['7', 'Apple', '$383,285', '-2.8%', '$96,995', '-2.8%', '$352,583', '161,000', '1', '22']
['8', 'UnitedHealth Group', '$371,622', '14.6%', '$22,381', '11.2%', '$273,720', '440,000', '2', '28']
['9', 'Berkshire Hathaway', '$364,482', '20.7%', '$96,223', '-', '$1,069,978', '396,500', '5', '28']
['10', 'CVS Health', '$357,776', '11%', '$8,344', '101.1%', '$249,728', '259,500', '1', '29']
['11', 'Volkswa

In [33]:
company_data = []  # Initialize an empty list to store company data

for company1 in soup.find_all('tbody', attrs={'role': 'rowgroup'}):
    for company2 in company1.find_all('tr', attrs={'role': 'row'}):
        details1 = company2.find_all('td', attrs={'role': 'cell', 'data-cy': 'list-cell'})

        if details1:
            # Extract and append the row data
            row_data = [cell.text.strip() for cell in details1]
            company_data.append(row_data)
        else:
            # Append a row of NaNs if no data is found
            company_data.append([np.NaN] * len(details1))

# Print the extracted data

print('The count of companies: ')
print(len(company_data))
print()
print(company_data)

The count of companies: 
500

[['1', 'Walmart', '$648,125', '6%', '$15,511', '32.8%', '$252,399', '2,100,000', '-', '30'], ['2', 'Amazon', '$574,785', '11.8%', '$30,425', '-', '$527,854', '1,525,000', '2', '16'], ['3', 'State Grid', '$545,947.5', '3%', '$9,204.3', '12.4%', '$781,126.2', '1,361,423', '-', '24'], ['4', 'Saudi Aramco', '$494,890.1', '-18%', '$120,699.3', '-24.1%', '$660,819.2', '73,311', '-2', '6'], ['5', 'Sinopec Group', '$429,699.7', '-8.8%', '$9,393.4', '-2.7%', '$382,688', '513,434', '1', '26'], ['6', 'China National Petroleum', '$421,713.6', '-12.7%', '$21,294.7', '1%', '$630,562', '1,026,301', '-1', '24'], ['7', 'Apple', '$383,285', '-2.8%', '$96,995', '-2.8%', '$352,583', '161,000', '1', '22'], ['8', 'UnitedHealth Group', '$371,622', '14.6%', '$22,381', '11.2%', '$273,720', '440,000', '2', '28'], ['9', 'Berkshire Hathaway', '$364,482', '20.7%', '$96,223', '-', '$1,069,978', '396,500', '5', '28'], ['10', 'CVS Health', '$357,776', '11%', '$8,344', '101.1%', '$249,728

In [35]:
company_data[0]

['1',
 'Walmart',
 '$648,125',
 '6%',
 '$15,511',
 '32.8%',
 '$252,399',
 '2,100,000',
 '-',
 '30']

In [37]:
company_data[2]

['3',
 'State Grid',
 '$545,947.5',
 '3%',
 '$9,204.3',
 '12.4%',
 '$781,126.2',
 '1,361,423',
 '-',
 '24']

In [39]:
column_headers = ['Rank',
                  'Name',
                  'Revenues ($M)',
                  'Revenue Percent Change',
                  'Profits ($M)',
                  'Profits Percent Change',
                  'Assets ($M)',
                  'Employees',
                  'Change in Rank',
                  'Years on Global 500 List']

Company_Data = [
                [company_data]
               ]

data_frame = pd.DataFrame(company_data, columns=cleaned_headers)
data_frame.head()

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,Change in Rank,Years on Global 500 List
0,1,Walmart,"$648,125",6%,"$15,511",32.8%,"$252,399","2,100,000",-,30
1,2,Amazon,"$574,785",11.8%,"$30,425",-,"$527,854","1,525,000",2,16
2,3,State Grid,"$545,947.5",3%,"$9,204.3",12.4%,"$781,126.2","1,361,423",-,24
3,4,Saudi Aramco,"$494,890.1",-18%,"$120,699.3",-24.1%,"$660,819.2","73,311",-2,6
4,5,Sinopec Group,"$429,699.7",-8.8%,"$9,393.4",-2.7%,"$382,688","513,434",1,26


## Data Cleaning

In [429]:
# Replacing the empty cells with NaN values using Numpy

data_frame1 = data_frame.replace('-', np.NaN)
data_frame1.head()

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,Change in Rank,Years on Global 500 List
0,1,Walmart,"$648,125",6%,"$15,511",32.8%,"$252,399","2,100,000",NaN,30
1,2,Amazon,"$574,785",11.8%,"$30,425",NaN,"$527,854","1,525,000",2,16
2,3,State Grid,"$545,947.5",3%,"$9,204.3",12.4%,"$781,126.2","1,361,423",NaN,24
3,4,Saudi Aramco,"$494,890.1",-18%,"$120,699.3",-24.1%,"$660,819.2","73,311",-2,6
4,5,Sinopec Group,"$429,699.7",-8.8%,"$9,393.4",-2.7%,"$382,688","513,434",1,26


In [45]:
data_frame1.shape

(500, 10)

In [49]:
data_frame1.size

5000

In [51]:
data_frame1.isnull().sum()

Rank                         0
Name                         0
Revenues ($M)                0
Revenue Percent Change       2
Profits ($M)                 0
Profits Percent Change      37
Assets ($M)                  0
Employees                    0
Change in Rank              55
Years on Global 500 List     0
dtype: int64

In [53]:
# Converting the data into CSV file

data_frame1.to_csv('Fortune_500_2024.csv', index = False)

In [7]:
# Reading the CSV file

data = pd.read_csv('Fortune_500 Raw Dataset.csv')
data.head()

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,Change in Rank,Years on Global 500 List
0,1,Walmart,"$648,125",6%,"$15,511",32.8%,"$252,399","2,100,000",NaN,30
1,2,Amazon,"$574,785",11.8%,"$30,425",NaN,"$527,854","1,525,000",2.0,16
2,3,State Grid,"$545,947.5",3%,"$9,204.3",12.4%,"$781,126.2","1,361,423",NaN,24
3,4,Saudi Aramco,"$494,890.1",-18%,"$120,699.3",-24.1%,"$660,819.2","73,311",-2.0,6
4,5,Sinopec Group,"$429,699.7",-8.8%,"$9,393.4",-2.7%,"$382,688","513,434",1.0,26


In [11]:
data.isnull().sum()

Rank                         0
Name                         0
Revenues ($M)                0
Revenue Percent Change       2
Profits ($M)                 0
Profits Percent Change      37
Assets ($M)                  0
Employees                    0
Change in Rank              55
Years on Global 500 List     0
dtype: int64

In [9]:
data.dtypes

Rank                          int64
Name                         object
Revenues ($M)                object
Revenue Percent Change       object
Profits ($M)                 object
Profits Percent Change       object
Assets ($M)                  object
Employees                    object
Change in Rank              float64
Years on Global 500 List      int64
dtype: object

In [356]:
# Profits Percent Change      37
# Change in Rank              55

In [13]:
# Dropping the columns which are not required

data.drop(columns = ['Profits Percent Change'], inplace = True)

In [15]:
data.drop(columns = ['Change in Rank'], inplace =  True)

In [17]:
data.head()

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
0,1,Walmart,"$648,125",6%,"$15,511","$252,399","2,100,000",30
1,2,Amazon,"$574,785",11.8%,"$30,425","$527,854","1,525,000",16
2,3,State Grid,"$545,947.5",3%,"$9,204.3","$781,126.2","1,361,423",24
3,4,Saudi Aramco,"$494,890.1",-18%,"$120,699.3","$660,819.2","73,311",6
4,5,Sinopec Group,"$429,699.7",-8.8%,"$9,393.4","$382,688","513,434",26


In [19]:
data.isnull().sum()

Rank                        0
Name                        0
Revenues ($M)               0
Revenue Percent Change      2
Profits ($M)                0
Assets ($M)                 0
Employees                   0
Years on Global 500 List    0
dtype: int64

In [21]:
data[data['Revenue Percent Change'].isnull()]

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
88,89,Rosneft Oil,"$107,543.1",NaN,"$14,870.4","$209,569.9","323,900",18
168,169,Lukoil,"$75,012.8",NaN,"$13,551.9","$95,935.4","107,596",24


In [23]:
# So, now we want to calculate the 'Revenue Percent Change' of those two companies and replace the calculated values with the empty ones

data.loc[(data['Name'] == 'Rosneft Oil'), 'Revenue Percent Change'] = 22.44

In [25]:

data.loc[(data['Name'] == 'Lukoil'), 'Revenue Percent Change'] = -40.06

In [27]:
data.isnull().sum()

Rank                        0
Name                        0
Revenues ($M)               0
Revenue Percent Change      0
Profits ($M)                0
Assets ($M)                 0
Employees                   0
Years on Global 500 List    0
dtype: int64

In [29]:
a = data[data['Name'] == 'Lukoil']
a

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
168,169,Lukoil,"$75,012.8",-40.06,"$13,551.9","$95,935.4","107,596",24


In [31]:
b = data[data['Name'] == 'Rosneft Oil']
b

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
88,89,Rosneft Oil,"$107,543.1",22.44,"$14,870.4","$209,569.9","323,900",18


In [123]:
# Now we need to remove all the special chars from all the rows

In [33]:
# Removing $ and , from Revenues Column
data['Revenues ($M)'] = [re.sub(r'[^\d]', '', str(row)) for row in data['Revenues ($M)']]

# Changing the data type of column
data['Revenues ($M)'] = data['Revenues ($M)'].astype(float)

In [35]:
# Removing the % symbol from 'Revenue Percent Change' column
data['Revenue Percent Change'] = [re.sub(r'[^\d.-]', '', str(row)) for row in data['Revenue Percent Change']]

# Changing the data type of the column to float as it is having float values
data['Revenue Percent Change'] = data['Revenue Percent Change'].astype(float)


In [297]:
# print(data['Profits ($M)'].unique())


In [39]:
# Ensure all values are strings
data['Profits ($M)'] = data['Profits ($M)'].astype(str)

In [41]:
# Removing '$' and ',' from Profits ($M) column
# Remove '$' and ',' from the strings
#data['Profits ($M)'] = data['Profits ($M)'].str.replace('[\$,]', '', regex=True)
data['Profits ($M)'] = data['Profits ($M)'].str.replace(r'[\$,]', '', regex=True)

# Changing the data type of the column to int
data['Profits ($M)'] = data['Profits ($M)'].astype(float)

In [51]:
# Check for any negative values in the column
# negative_values = data[data['Profits ($M)'] < 0]
# negative_values


In [57]:
# Removing $ and , from Assets ($M) column

data['Assets ($M)'] = data['Assets ($M)'].replace(r'[\$,]', '', regex = True)

# Changing the datatype of the column
data['Assets ($M)'] = data['Assets ($M)'].astype(float)

In [59]:
# Removing the commas from Employees columns
data['Employees'] = data['Employees'].replace(r'[\,]', '', regex = True)

# Changing the datatype of Employees column
data['Employees'] = data['Employees'].astype(int)

In [61]:
# Changing the 'Years on Global 500 List' to int type
data['Years on Global 500 List'] = data['Years on Global 500 List'].astype(int)

In [93]:
data

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
0,1,Walmart,648125.0,6.0,15511.0,252399.0,2100000,30
1,2,Amazon,574785.0,11.8,30425.0,527854.0,1525000,16
2,3,State Grid,5459475.0,3.0,9204.3,781126.2,1361423,24
3,4,Saudi Aramco,4948901.0,-18.0,120699.3,660819.2,73311,6
4,5,Sinopec Group,4296997.0,-8.8,9393.4,382688.0,513434,26
...,...,...,...,...,...,...,...,...
495,496,Air France-KLM Group,324523.0,16.9,1009.7,38093.7,76271,25
496,497,Enbridge,323495.0,-21.0,4588.3,136769.6,12450,11
497,498,ABB,32235.0,9.5,3745.0,40940.0,107900,29
498,499,Mitsubishi Heavy Industries,322232.0,3.8,1536.2,41371.9,77697,30


In [95]:
data.dtypes

Rank                          int64
Name                         object
Revenues ($M)               float64
Revenue Percent Change      float64
Profits ($M)                float64
Assets ($M)                 float64
Employees                     int32
Years on Global 500 List      int32
dtype: object

In [97]:
data.isnull().sum()

Rank                        0
Name                        0
Revenues ($M)               0
Revenue Percent Change      0
Profits ($M)                0
Assets ($M)                 0
Employees                   0
Years on Global 500 List    0
dtype: int64

In [99]:
data.shape

(500, 8)

In [101]:
data.size

4000

In [423]:
# Now we can see that the data is cleaned and well organized

In [103]:
# Converting the data to CSV file
data.to_csv('Fortune_500.csv', index = False)

In [105]:
import pandas as pd

In [107]:
data1 = pd.read_csv('Fortune_500.csv')
data1

,Rank,Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List
0,1,Walmart,648125.0,6.0,15511.0,252399.0,2100000,30
1,2,Amazon,574785.0,11.8,30425.0,527854.0,1525000,16
2,3,State Grid,5459475.0,3.0,9204.3,781126.2,1361423,24
3,4,Saudi Aramco,4948901.0,-18.0,120699.3,660819.2,73311,6
4,5,Sinopec Group,4296997.0,-8.8,9393.4,382688.0,513434,26
...,...,...,...,...,...,...,...,...
495,496,Air France-KLM Group,324523.0,16.9,1009.7,38093.7,76271,25
496,497,Enbridge,323495.0,-21.0,4588.3,136769.6,12450,11
497,498,ABB,32235.0,9.5,3745.0,40940.0,107900,29
498,499,Mitsubishi Heavy Industries,322232.0,3.8,1536.2,41371.9,77697,30


### Adding a new column to know the Revenue Generated by each company in 2024

In [5]:
# Calculate the percentage change amount for each row
data1['Revenue Growth/Decline ($M)'] = data1['Revenues ($M)'] * (data1['Revenue Percent Change'] / 100)


In [150]:
data1 = data1.rename(columns={'Revenue_generated_in_2024':'Revenue_generated_in_2024 ($M)'})

In [127]:
data1 = data1.rename(columns={'Name':'Company_Name'})

In [143]:
data1.drop(columns = ['Revenue_Percent_Change_Amount ($M)'], inplace = True)

In [9]:
data1.drop(columns = ['Revenue_generated_in_2024 ($M)'], inplace = True)

In [11]:
data1

,Rank,Company_Name,Revenues ($M),Revenue Percent Change,Profits ($M),Assets ($M),Employees,Years on Global 500 List,Revenue Growth/Decline ($M)
0,1,Walmart,648125.0,6.0,15511.0,252399.0,2100000,30,38887.500
1,2,Amazon,574785.0,11.8,30425.0,527854.0,1525000,16,67824.630
2,3,State Grid,5459475.0,3.0,9204.3,781126.2,1361423,24,163784.250
3,4,Saudi Aramco,4948901.0,-18.0,120699.3,660819.2,73311,6,-890802.180
4,5,Sinopec Group,4296997.0,-8.8,9393.4,382688.0,513434,26,-378135.736
...,...,...,...,...,...,...,...,...,...
495,496,Air France-KLM Group,324523.0,16.9,1009.7,38093.7,76271,25,54844.387
496,497,Enbridge,323495.0,-21.0,4588.3,136769.6,12450,11,-67933.950
497,498,ABB,32235.0,9.5,3745.0,40940.0,107900,29,3062.325
498,499,Mitsubishi Heavy Industries,322232.0,3.8,1536.2,41371.9,77697,30,12244.816


In [13]:
# Converting the data to CSV file
data1.to_csv('Fortune_500.csv', index = False)